In [1]:
import numpy as np

In [7]:
info_path = "../data/scene_info/0229.npz"
data = np.load(info_path, allow_pickle=True)
data.files

['image_paths',
 'depth_paths',
 'intrinsics',
 'poses',
 'overlap_matrix',
 'scale_ratio_matrix',
 'angles',
 'n_points3D',
 'points3D_id_to_2D',
 'points3D_id_to_ndepth']

In [8]:
gt_overlap = data["overlap_matrix"]

(318, 318)

In [9]:
overlap_matrix_path = "../data/0229/metrics/baseline/sparse-overlap-2023-03-26_18-17-41.npy"
overlap_matrix = np.load(overlap_matrix_path, allow_pickle=True)

(318, 318)

In [11]:
print(f"GT overlap shape:  {gt_overlap.shape}")
print(f"Our overlap shape: {overlap_matrix.shape}")
print(f"GT mean overlap:   {np.mean(gt_overlap[gt_overlap > 0]):.4f}")
print(f"Our mean overlap:  {np.mean(overlap_matrix[overlap_matrix > 0]):.4f}")

GT overlap shape:  (318, 318)
Our overlap shape: (318, 318)
GT mean overlap:   0.1346
Our mean overlap:  0.1888
